In [1]:
# Import libraries
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

%load_ext lab_black

In [2]:
# Create new spider class
class WikiSpider(scrapy.Spider):
    name = "WS"

    # Insert API call
    start_urls = [
        "https://en.wikipedia.org/w/api.php?action=query&format=xml&prop=linkshere&titles=Monty_Python&lhprop=title%7Credirect"
    ]

    # Use xpath to extract information
    def parse(self, response):
        for item in response.xpath("//lh"):
            # Identify the type of page the link comes from: '0' means it is a Wikipedia entry
            # Only keep Wiki entries
            if item.xpath("@ns").extract_first() == "0":
                yield {"title": item.xpath("@title").extract_first()}
        # Get info needed to continue to the next 10 entries
        next_page = response.xpath("continue/@lhcontinue").extract_first()

        # Recursively call the spider to process the next ten entries, if they exist
        if next_page is not None:
            next_page = "{}&lhcontinue={}".format(self.start_urls[0], next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [3]:
process = CrawlerProcess(
    {
        "FEED_FORMAT": "json",
        "FEED_URI": "PythonLinks.json",
        # robots.txt file doesn't apply to API
        "ROBOTSTXT_OBEY": False,
        "USER_AGENT": "ThinkfulDataScienceBootcampCrawler (thinkful.com)",
        "AUTOTHROTTLE_ENABLED": True,
        "HTTPCACHE_ENABLED": True,
        "LOG_ENABLED": False,
        # Limit our scraper to the first 100 links
        "CLOSESPIDER_PAGECOUNT": 10,
    }
)


# Starting the crawler with our spider.
process.crawl(WikiSpider)
process.start()
print("First 100 links extracted!")

First 100 links extracted!


In [4]:
# Checking whether we got data
Monty = pd.read_json("PythonLinks.json", orient="records")
print(Monty.shape)
Monty.tail()

(93, 1)


,title
88,Catherine Deneuve
89,Beyond the Fringe
90,Cunt
91,List of Welsh people
92,"Downing College, Cambridge"
